# Zenodo

There are two types of Zenodo interfaces. One interfaces to the public repositories (`ZenodoRecord`), the other is for testing and accessed the sandbox server (`ZenodoSandboxDeposit`).

The class diagram below shows how they are constructed. First, an abstract zenodo interface class (`AbstractZenodoInterface`) is derived. From this, the concrete interface classes are derived.

<img src="../../_static/repo_class_diagram.svg"
     alt="../../_static/repo_class_diagram.svg"
     style="margin-right: 10px; height: 500px;" />
     

## Example usage

The example below will upload an HDF file to the sandbox server:

In [1]:
from h5rdmtoolbox.repository import zenodo
import h5rdmtoolbox as h5tbx

Failed to import module h5tbx


Init a Repo:

In [2]:
repo = zenodo.ZenodoSandboxDeposit(None)

Create a test file:

In [3]:
with h5tbx.File() as h5:
    h5.create_dataset('velocity', shape=(10, 30), attrs={'units': 'm/s'})
filename = h5.hdf_filename

Prepare metadata

In [4]:
from h5rdmtoolbox.repository.zenodo import metadata
from datetime import datetime

meta = metadata.Metadata(
    version="0.1.0-rc.1+build.1",
    title='[deleteme]h5tbxZenodoInterface',
    description='A toolbox for managing HDF5-based research data management',
    creators=[metadata.Creator(name="Probst, Matthias",
                      affiliation="KIT - ITS",
                      orcid="0000-0001-8729-0482")],
    contributors=[metadata.Contributor(name="Probst, Matthias",
                              affiliation="KIT - ITS",
                              orcid="0000-0001-8729-0482",
                              type="ContactPerson")],
    upload_type='image',
    image_type='photo',
    access_right='open',
    keywords=['hdf5', 'research data management', 'rdm'],
    publication_date=datetime.now(),
    embargo_date='2020'
)

Add (and publish) the metadata:

In [5]:
repo.metadata = meta

Upload the HDF5 file. Note, that there's a difference between `upload_file` and `upload_hdf_file`. The latter not just uploads the HDF5 file but also generates a metadata file and uploads it, too. The reason behind it is, that the HDF5 file can be very large. Before downloading the (large) HDF5 file, the user may want to review the metadata content first. This is possible through the "metamapper-function" which needs to be provided during upload.

In [13]:
import pathlib
from h5rdmtoolbox import jsonld
class HDF_to_JSONLD:
    def __init__(self, skipND):
        self.skipND = skipND
        
    def __call__(self, hdf_filename):
        json_filename = pathlib.Path(hdf_filename).with_suffix('.json')
        with open(json_filename, 'w') as f:
            f.write(jsonld.dump_file(hdf_filename, skipND=self.skipND))
        return json_filename

In [14]:
repo.upload_hdf_file(filename, metamapper=HDF_to_JSONLD(1))

C:\Users\da4323\Documents\programming\GitHub\h5RDMtoolbox\h5rdmtoolbox\repository\zenodo\core.py:325: UserWarning: Filename "C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\tmp\tmp_4\tmp0.hdf" already exists in deposit. Skipping..."
  warnings.warn(f'Filename "{filename}" already exists in deposit. Skipping..."', UserWarning)


List the just uploaded files, by requesting the current filenames in the repository:

In [16]:
repo.get_filenames()

{'tmp0.hdf': {'id': 'b0fd97a9-8a6d-4fab-b7f3-6f7903f2926f',
  'filename': 'tmp0.hdf',
  'filesize': 12384,
  'checksum': '02d007d9b6d35f7b35a7b47a30c4efa1',
  'links': {'self': 'https://sandbox.zenodo.org/api/deposit/depositions/38652/files/b0fd97a9-8a6d-4fab-b7f3-6f7903f2926f',
   'download': 'https://sandbox.zenodo.org/api/records/38652/draft/files/tmp0.hdf/content'}},
 'tmp0.json': {'id': '6910f6b4-120f-4e66-877e-e1427c9d5dd3',
  'filename': 'tmp0.json',
  'filesize': 1879,
  'checksum': '9e0e31878c459bb61755b3026b3fb2ce',
  'links': {'self': 'https://sandbox.zenodo.org/api/deposit/depositions/38652/files/6910f6b4-120f-4e66-877e-e1427c9d5dd3',
   'download': 'https://sandbox.zenodo.org/api/records/38652/draft/files/tmp0.json/content'}}}